In [ ]:
RESULT_DIR = "../out/2025-05-05_13-29-23_librispeech-pc-test-clean_large-v3-turbo"
# RESULT_DIR = "/home/niko/Downloads/results"

In [ ]:
import os
import sys
sys.path.append("..")

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from src.eval.SampleResult import SampleResult

In [ ]:
files = os.listdir(RESULT_DIR)
files = [f for f in files if f.endswith("final.json")]

# get ids from the first part of the filename separated by "_"
file_ids = [f.split("_")[0] for f in files]

samples = [SampleResult.load_by_id(RESULT_DIR, file_id) for file_id in file_ids]

In [ ]:
df = pd.DataFrame()

In [ ]:
df.sort_values(by=["alignment_sequence", "temporal_alignment_tolerance"], inplace=True)
df

# Evaluation Parameters

In [ ]:
ALIGNMENT_SEQUENCE = "final"
TEMPORAL_ALIGNMENT_TOLERANCE = 0.1
ALIGNMENT_WORD_NORMALIZATION = True     # No difference

In [ ]:
eval = pd.Series()
eval["num_samples"] = len(samples)
eval["alignment_sequence"] = ALIGNMENT_SEQUENCE
eval["alignment_word_normalization"] = ALIGNMENT_WORD_NORMALIZATION
eval["temporal_alignment_tolerance"] = TEMPORAL_ALIGNMENT_TOLERANCE

## Alignments

Build the alignments relative to either the `final` sequence of the online, or the `baseline` with offline transcription.

In [ ]:
_alignment_sequences = [s.final if ALIGNMENT_SEQUENCE == "final" else s.baseline for s in samples]

for sample in samples:
    sample.build_alignments(normalize_words=ALIGNMENT_WORD_NORMALIZATION,
                            align_to=ALIGNMENT_SEQUENCE,
                            temporal_tolerance=TEMPORAL_ALIGNMENT_TOLERANCE)

In [ ]:
samples_df = pd.DataFrame([{
    "id": sample.sample_id,
    "sample": sample,
    "word_count_final": len(sample.final),
    "word_count_baseline": len(sample.baseline),
    "word_count_transcript": len(sample.transcript),
} for sample in samples])
samples_df

In [ ]:
def total_partial_words(sample: SampleResult) -> int:
    partial_words = [len(partial.result) for partial in sample.partials]
    return sum(partial_words)

def unaligned_partial_words(sample: SampleResult) -> int:
    unalignments = [len(alignements.unalignments) for alignements in sample.alignments]
    return sum(unalignments)

samples_df["unaligned_partial_count"] = samples_df["sample"].apply(unaligned_partial_words)
samples_df["partial_count"] = samples_df["sample"].apply(total_partial_words)

eval["total_partial_count"] = samples_df["partial_count"].sum().item()
eval["unaligned_partial_count"] = samples_df["unaligned_partial_count"].sum().item()

## Word first correct

The word first correct (wfc) is defined as the first correct occurence of a word.

In [ ]:
from src.eval.metrics.word_first_correct import word_first_correct_response
wfc = [[word_first_correct_response(_alignment_sequences[j], s.partials, i, s.alignments) for i in range(len(_alignment_sequences[j]))] for j, s in enumerate(samples)]

In [ ]:
total_wfc = np.concat(wfc)
wfc_latency = np.array([r["latency"] for r in total_wfc if r is not None])
none_wfc_count = len(total_wfc) - len(wfc_latency)
print("None values in wfc: ", none_wfc_count, f"({none_wfc_count / len(total_wfc) * 100:.2f}%)")
eval["wfc_none_count"] = none_wfc_count

In [ ]:
print("Min:", np.min(wfc_latency))
print("Max:", np.max(wfc_latency))
print("Mean:", np.mean(wfc_latency))
print("Median:", np.median(wfc_latency))
print("Std:", np.std(wfc_latency))
print("P95:", np.percentile(wfc_latency, 95))
print("P99:", np.percentile(wfc_latency, 99))

eval["wfc_latency_min"] = np.min(wfc_latency).item()
eval["wfc_latency_max"] = np.max(wfc_latency).item()
eval["wfc_latency_mean"] = np.mean(wfc_latency).item()
eval["wfc_latency_std"] = np.std(wfc_latency).item()
eval["wfc_latency_median"] = np.median(wfc_latency).item()
eval["wfc_latency_negative_count"] = len([x for x in wfc_latency if x < 0])
eval["wfc_latency_p95"] = np.percentile(wfc_latency, 95).item()
eval["wfc_latency_p99"] = np.percentile(wfc_latency, 99).item()
eval["wfc_latency_count"] = len(wfc_latency)

In [ ]:
plt.hist(wfc_latency,
         bins=100,
         range=(math.floor(np.percentile(wfc_latency, 1)), math.ceil(np.percentile(wfc_latency, 99).item()))
         )
plt.xlabel("Latency (s)")
plt.ylabel("Count")
plt.title(f"Word First Correct (WFC) Latency Distribution, {ALIGNMENT_SEQUENCE} alignment")
plt.grid()

## Word first final

The word first final (wff) is defined as the first occurence after which a word will never change to a different prediction.

In [ ]:
from src.eval.metrics.word_first_final import word_first_final_response
wff = [[word_first_final_response(_alignment_sequences[j], s.partials, i, s.alignments) for i in range(len(_alignment_sequences[j]))] for j, s in enumerate(samples)]

In [ ]:
total_wff = np.concat(wff)
wff_latency = np.array([r["latency"] for r in total_wff if r is not None])
none_wff_count = len(total_wff) - len(wff_latency)
print("None values in wff: ", none_wff_count, f"({none_wff_count / len(total_wff) * 100:.2f}%)")
eval["wff_none_count"] = none_wff_count

In [ ]:
print("Min:", np.min(wff_latency))
print("Max:", np.max(wff_latency))
print("Mean:", np.mean(wff_latency))
print("Median:", np.median(wff_latency))
print("Std:", np.std(wff_latency))
print("P95:", np.percentile(wff_latency, 95))
print("P99:", np.percentile(wff_latency, 99))

eval["wff_latency_min"] = np.min(wff_latency).item()
eval["wff_latency_max"] = np.max(wff_latency).item()
eval["wff_latency_mean"] = np.mean(wff_latency).item()
eval["wff_latency_std"] = np.std(wff_latency).item()
eval["wff_latency_median"] = np.median(wff_latency).item()
eval["wff_latency_negative_count"] = len([x for x in wff_latency if x < 0])
eval["wff_latency_p95"] = np.percentile(wff_latency, 95).item()
eval["wff_latency_p99"] = np.percentile(wff_latency, 99).item()
eval["wff_latency_count"] = len(wff_latency)

In [ ]:
plt.hist(wff_latency, bins=100, range=(math.floor(np.percentile(wff_latency, 1)), math.ceil(np.percentile(wff_latency, 99).item())))
plt.xlabel("Latency (s)")
plt.ylabel("Count")
plt.title(f"Word First Final (WFF) Latency Distribution, {ALIGNMENT_SEQUENCE} alignment")
plt.grid()

In [ ]:
df = pd.concat([df, eval.to_frame().T], ignore_index=True)

## Visualize Alignment

In [ ]:
id = "7127-75946-0000"
sample = next(s for s in samples if s.sample_id == id)

In [ ]:
from matplotlib.patches import Rectangle

DRAW_CONFIRMED_ALIGNMENTS = False

_alignment_sequence = sample.final if ALIGNMENT_SEQUENCE == "final" else sample.baseline

window_length = 30
window_start = 473
window_end = window_start + window_length

words = _alignment_sequence[window_start:window_end]

window_start_t = words[0].start
window_end_t = words[-1].end

# Filter predictions and keep original indices
indexed_predictions = [
    (i, p) for i, p in enumerate(sample.partials)
    if window_start_t <= p.window[1] and (p.observation_time <= window_end_t or p.window[1] <= window_end_t)
]
indexed_predictions = indexed_predictions[::-1]  # Reverse for display

# Final word mapping (global indices to x/y coordinates)
final_word_positions = {
    i: {
        "x": (word.start + word.end) / 2,
        "y": len(indexed_predictions) + 0.25  # fixed y for final row
    }
    for i, word in enumerate(_alignment_sequence)
}

# Determine max observation time for x-axis limit
max_obs_time = max(window_end_t, max(p.observation_time for _, p in indexed_predictions))

fig, ax = plt.subplots(figsize=(30, 8))

for row_index, (original_index, partial) in enumerate(indexed_predictions):
    win_start, win_end = partial.window
    obs_time = partial.observation_time

    # Green prediction window
    ax.add_patch(Rectangle((win_start, row_index), win_end - win_start, 1, color="green", alpha=0.3))

    marker_width = 0.01
    ax.add_patch(Rectangle((win_end - marker_width / 2, row_index), marker_width, 1, color="green", alpha=0.9))

    # Red delay box
    if obs_time > win_end:
        delay = obs_time - win_end
        ax.add_patch(Rectangle(
            (win_end, row_index), delay, 1,
            color="red", alpha=0.2, hatch='////', linewidth=0.5, fill=True
        ))
        # Delay label
        ax.text(win_end + delay / 2, row_index + 0.5, f"{delay:.2f}s", ha="center", va="center", fontsize=10, color="red")

    ax.add_patch(Rectangle((obs_time - marker_width / 2, row_index), marker_width, 1, color="red", alpha=0.9))

    # Word boxes
    for word_index, word in enumerate(partial.result):
        if window_start_t <= word.start <= window_end_t:
            ax.add_patch(Rectangle(
                (word.start, row_index + 0.25), word.end - word.start, 0.5,
                edgecolor="darkgreen", facecolor="none", linewidth=1
            ))
            ax.text((word.start + word.end) / 2, row_index + 0.5, word.word,
                    ha="center", va="center", fontsize=9, color="darkgreen")
            
            # Draw alignment line
            wa = None
            accepted = False
            if DRAW_CONFIRMED_ALIGNMENTS:
                for _wa in sample.alignments[original_index].confirmed_alignments:
                    if _wa.partial_word_index == word_index:
                        wa = _wa
                        break
            for _wa in sample.alignments[original_index].accepted_alignments:
                if _wa.partial_word_index == word_index:
                    wa = _wa
                    accepted = True
                    break
            if wa is None and not DRAW_CONFIRMED_ALIGNMENTS:
                for _wa in sample.alignments[original_index].potential_alignments:
                    if _wa.partial_word_index == word_index:
                        wa = _wa
                        break
            if wa is not None:
                if wa.final_word_index in final_word_positions:
                    x0 = (word.start + word.end) / 2
                    y0 = row_index + 0.75
                    x1 = final_word_positions[wa.final_word_index]["x"]
                    y1 = final_word_positions[wa.final_word_index]["y"]
                    color = "blue" if DRAW_CONFIRMED_ALIGNMENTS else "lightgreen" if accepted else "orange"
                    ax.plot([x0, x1], [y0, y1], color=color, linestyle="dashed", linewidth=1)
            else:
                ax.plot((word.start + word.end) / 2, row_index + 0.85, "o", color="red", markersize=3, alpha=0.5)
            

# Draw top word layer (blue)
word_y = len(indexed_predictions)
for i, word in enumerate(words):
    start = word.start
    end = word.end
    ax.add_patch(Rectangle((start, word_y), end - start, 1, color="blue", alpha=0.5))
    ax.text((start + end) / 2, word_y + 0.5, word.word, ha="center", va="center", fontsize=10)
    ax.text((start + end) / 2, word_y + 1.3, f"{window_start + i}", ha="center", va="center", fontsize=10, color="black")
    ax.axvline(start, color="black", linestyle="--", linewidth=0.5, alpha=0.5)
    ax.axvline(end, color="black", linestyle="--", linewidth=0.5, alpha=0.5)

window_end_t = max_obs_time + 0.5

# Axes setup
ax.set_xlim(window_start_t, window_end_t)
ax.set_ylim(0, word_y + 2)
ax.set_yticks([i + 0.5 for i in range(len(indexed_predictions))])
ax.set_yticklabels([str(idx) for idx, _ in indexed_predictions])
ax.xaxis.tick_top()

plt.show()

# Automated alignment evaluation

In [418]:
from tqdm import tqdm

In [419]:
ALIGNMENT_SEQUENCES = ["final", "baseline"]
TEMPORAL_ALIGNMENT_TOLERANCES = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

In [ ]:
metrics_df = pd.DataFrame()

for ALIGNMENT_SEQUENCE in ALIGNMENT_SEQUENCES:
    print(f"Evaluating {ALIGNMENT_SEQUENCE} alignments")
    for TEMPORAL_ALIGNMENT_TOLERANCE in tqdm(TEMPORAL_ALIGNMENT_TOLERANCES):
        _alignment_sequences = [s.final if ALIGNMENT_SEQUENCE == "final" else s.baseline for s in samples]

        for sample in samples:
            sample.build_alignments(normalize_words=ALIGNMENT_WORD_NORMALIZATION,
                                    align_to=ALIGNMENT_SEQUENCE,
                                    temporal_tolerance=TEMPORAL_ALIGNMENT_TOLERANCE)
        
        eval = pd.Series()
        eval["alignment_sequence"] = ALIGNMENT_SEQUENCE
        eval["temporal_alignment_tolerance"] = TEMPORAL_ALIGNMENT_TOLERANCE

        eval["unaligned_partial_words"] = sum([unaligned_partial_words(s) for s in samples])

        wfc = [[word_first_correct_response(_alignment_sequences[j], s.partials, i, s.alignments) for i in range(len(_alignment_sequences[j]))] for j, s in enumerate(samples)]
        total_wfc = np.concat(wfc)
        eval["wfc_latency"] = np.array([r["latency"] for r in total_wfc if r is not None])

        wff = [[word_first_final_response(_alignment_sequences[j], s.partials, i, s.alignments) for i in range(len(_alignment_sequences[j]))] for j, s in enumerate(samples)]
        total_wff = np.concat(wff)
        eval["wff_latency"] = np.array([r["latency"] for r in total_wff if r is not None])

        metrics_df = pd.concat([metrics_df, eval.to_frame().T], ignore_index=True)

metrics_df.sort_values(by=["alignment_sequence", "temporal_alignment_tolerance"], inplace=True)
metrics_df.info()


100%|██████████| 2/2 [06:25<00:00, 192.72s/it]


KeyError: 'alignment_sequence'

In [421]:
metrics_df

,unaligned_partial_words,wfc_latency,wff_latency
0,5600,"[1.041731370911002, 0.7617313709110021, 11.388...","[15.973553719297051, 15.693553719297052, 15.57..."
1,9021,"[2.121731370911002, 0.6341608881205321, 0.4941...","[2.121731370911002, 0.6341608881205321, 0.4941..."
2,8228,"[2.121731370911002, 0.6341608881205321, 0.4941...","[2.121731370911002, 0.6341608881205321, 0.4941..."
3,7712,"[1.114160888120532, 0.6341608881205321, 0.4941...","[1.114160888120532, 0.6341608881205321, 0.4941..."
4,7335,"[1.114160888120532, 0.6341608881205321, 0.4941...","[1.114160888120532, 0.6341608881205321, 0.4941..."
5,7133,"[1.114160888120532, 0.6341608881205321, 0.4941...","[1.114160888120532, 0.6341608881205321, 0.4941..."
6,2729,"[1.035239720568061, 0.7952397205680608, 0.6552...","[11.867625450566411, 16.74449127547443, 16.604..."
7,6813,"[2.121731370911002, 0.6341608881205321, 0.4941...","[2.121731370911002, 0.6341608881205321, 0.4941..."
8,5476,"[2.121731370911002, 0.6341608881205321, 0.4941...","[2.121731370911002, 0.6341608881205321, 0.4941..."
9,4715,"[1.114160888120532, 0.6341608881205321, 0.4941...","[1.114160888120532, 0.6341608881205321, 0.4941..."
